In [2]:
from Connect4 import Connect4
from Robots import Robots
from GamePlay import GamePlay
from tensorflow.keras.models import Sequential, save_model, load_model
import numpy as np
import random

import datetime


import keras
from keras import layers
from keras import models

In [83]:
#class that creates the connect4 environment
class DeepQLAgent():

    def __init__(self,player=1,gameplay=GamePlay(),memory=None):
        self.player=player
        
        self.gameplay=gameplay
        self.ROWS = self.gameplay.ROWS
        self.COLUMNS = self.gameplay.COLUMNS
        
        self.batch_size = 100
        self.lr=.01
        self.INPUT_SHAPE=(self.ROWS, self.COLUMNS, 1)
        self.action_size =self.COLUMNS
        self.gamma=.5
        self.epsilon = 1.0 #starting explore probability
        self.explore_stop = .01 #stopping explore rate
        self.decay_rate = 0.00001 #decay rate # 0.00001

        self.memory_size = 5000 # number of experiences to keep
        self.memory_start = 200 # starting size of memory bank
        self.memory=self._memory_initiate()
        self.avgrewards=[]
        self.model=self._build_model()
        self.losses=[]

    #build model
    def _build_model(self):
        model=models.Sequential()
        
        model.add(layers.Flatten(input_shape=self.INPUT_SHAPE))
        #model.add(layers.Dense(128, activation = 'relu'))
        #model.add(layers.Dense(100, activation = 'relu'))
        model.add(layers.Dense(64, activation = 'relu'))
        model.add(layers.Dense(32, activation = 'relu'))
        #model.add(layers.Dense(16, activation = 'relu'))
        model.add(layers.Dense(self.action_size,activation='linear'))

        '''
        model.add(layers.Conv2D(4, kernel_size = (4, 4), activation='relu',
                                padding='same',strides=(2,2), input_shape=self.INPUT_SHAPE))
        #model.add(layers.Dropout(0.25))
        model.add(layers.Flatten()) 
        model.add(layers.Dense(self.action_size,activation='softmax'))'''

        optimizer = keras.optimizers.Adam(lr=self.lr)
        model.compile(loss='mse', optimizer=optimizer)
        
        return model


    def _memory_initiate(self):
        memory=[]
        turn = 0
        #player=1
        status = None
        while len(memory)<self.memory_start:
            if status != 'Keep Playing!': #if we are in a terminal state restart game
                self.gameplay.reset()
                turn = 0

            state = self.gameplay.BOARD.copy()
            actions = self.gameplay.Get_Legal_Moves(self.gameplay.BOARD)
            action = np.random.choice(actions,1)[0]
            #take random actions. record their states,actions, rewards, next states, and status

            #player1
            if turn % 2 ==0:
                self.gameplay.Add_Piece(1,action,self.gameplay.BOARD)
                #turn +=1

            #player2
            elif turn % 2 ==1:
                self.gameplay.Add_Piece(2,action,self.gameplay.BOARD)
                #turn +=1


            status = self.gameplay.Check_Goal(self.gameplay.BOARD)
            if status !='Keep Playing!':
                state_prime = np.zeros((self.gameplay.ROWS,self.gameplay.COLUMNS))
                state_prime[state_prime < 1] = self.player
            else:
                state_prime = self.gameplay.BOARD.copy() #gamestate
            reward = self.gameplay.Get_Score(self.player,state_prime)

            if turn % 2 ==0 and self.player==1:
                state_prime = self.gameplay.BOARD.copy()
                reward = self.gameplay.Get_Score(self.player,state_prime)
                memory.append((state,action,reward,state_prime,status)) #memorize this
            if turn % 2 ==1 and self.player==2:
                state_prime = self.gameplay.BOARD.copy()
                reward = self.gameplay.Get_Score(self.player,state_prime)
                memory.append((state,action,reward,state_prime,status)) #memorize this
            turn +=1
        return memory    
    

    #function to hold previous states/actions/rewards/nextstates/status
    def memorize(self,state,action,reward,state_prime,done):
        if len(self.memory)>=self.memory_size: #if memory is full remove random element before adding new one
            self.memory.pop(random.randrange(len(self.memory)))  
        self.memory.append((state,action,reward,state_prime,done)) #append memory

    def load(self,name):
        self.model.load_weights(name)

    def save(self,name):
        self.model.save_weights(name)
    

    #predict values of state using model
    #I hate shapes/shaping with numpy/keras; can never get it to work so this is a function to reshape it for me
    def model_predict(self,state):
        state_reshape=np.expand_dims(np.expand_dims(state, axis=3),axis=0)
        return self.model.predict(state_reshape)
    
    #function to take action given state                  
    def make_move(self,state): 
        if np.random.rand() <=self.epsilon:
            #explore
            #print('explore, epsilon is',self.epsilon)
            legal_actions=self.gameplay.Get_Legal_Moves(state)
            return random.randrange(len(legal_actions))
        else:
            #don't explore
            qvals = self.model_predict(state)
            return np.argmax(qvals)


    def replay(self,batch_size):
        batch = random.sample(self.memory,self.batch_size)
        for el in batch:
            if self.player==1: #replace 2's with -1s
                el[0][el[0]>1]=-1
                el[3][el[3]>1]=-1
            if self.player==2: #replace 1s with -1s. replace 2's with 1s
                el[0][(el[0] < 2) & (el[0] > 0)] = -1
                el[0][el[0]>1]=1                
                el[3][(el[3] < 2) & (el[3] > 0)] = -1
                el[3][el[3]>1]=1                
        
        states = []
        targets_f = []
        #working
        #for state,action, reward, state_prime,status in batch:
        #    qs=self.model_predict(state)[0]
        #    legal=gameplay.Get_Legal_Moves(state)
        #    #update qs... i think
        #    for i in range(len(qs)): 
        #        sp=gameplay.Add_Piece(self.player,i,state)
        #        r=gameplay.get_reward(self.player,sp)
        #        newstatus=gameplay.Check_Goal(sp)
        #        if i not in legal: #if not a legal move
        #            qs[i]=r #q equals reward
        #        elif newstatus != 'Keep Playing!' or status != 'Keep Playing!': #if terminal status
        #            qs[i]=r #q equals reward
        #        elif action==i: #otherwise, do the q update
        #            qs[action]=(reward + self.gamma * np.argmax(self.model_predict(state_prime)[0]))
        #    
        #    targets_f.append(qs)
        #    states.append(state)
                           
        
        for state,action, reward, state_prime,status in batch:
            legal=gameplay.Get_Legal_Moves(state)
            if status != 'Keep Playing!':
                target = reward 
            elif status == 'Keep Playing!':
                if action not in gameplay.Get_Legal_Moves(state): #if not a legal move make target the negative reward
                    target = reward
                else:
                    target = (reward + self.gamma * np.argmax(self.model_predict(state_prime)[0])) #idk why they have first element.
            
            self.avgrewards.append(reward)
            target_f = self.model_predict(state_prime)
            target_f[0][action]=target #update target value
            for i in range(len(target_f[0])):
                if i not in legal:
                    target_f[0][i]=min(target_f[0])
                #if
            targets_f.append(target_f[0])
            states.append(state)
            
        states=np.expand_dims(np.array(states), axis=3) #reshaping to train model
        targets_f=np.array(targets_f)

        history = self.model.fit(states,targets_f,epochs=1,verbose=0)
        loss = history.history['loss'][0]
        self.losses.append(loss)
        if self.epsilon <= self.explore_stop:
            self.epsilon=self.explore_stop
        else:
            self.epsilon = self.epsilon-self.decay_rate
        #print('epsilon:',self.epsilon)



In [84]:
gameplay=GamePlay()
agent=DeepQLAgent()
agent.replay(agent.batch_size)

agent2=DeepQLAgent(player=2)
agent2.replay(agent.batch_size)


from Robots import Robots
bot=Robots(depth=1)
#bot.MiniMaxAlphaBeta_bot(state)
agent.model.summary()

C:\Users\wamsa\Anaconda3\lib\site-packages\ipykernel_launcher.py:114: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_48 (Flatten)         (None, 42)                0         
_________________________________________________________________
dense_90 (Dense)             (None, 64)                2752      
_________________________________________________________________
dense_91 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_92 (Dense)             (None, 7)                 231       
Total params: 5,063
Trainable params: 5,063
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#secondgo
EPISODES = 200000
episode=0
threshold=60

#batch_size=50
turn=0
p1wins=0
p2wins=0

p1prev=0
p2prev=0

agent.epsilon=.5

while episode<EPISODES:

    #player1
    if turn % 2 ==0:
        state=gameplay.BOARD.copy() #get state
        action=agent.make_move(state) #get action
        gameplay.Add_Piece(1,action,gameplay.BOARD) #do action
        status=gameplay.Check_Goal(gameplay.BOARD) #get status

    #if it's player 1's turn and player 1 wins, 
    #state prime/reward is immediately after action instead after the other player's action
        if status == 'Player 1 wins!':
            state_prime = gameplay.BOARD.copy()
            reward = gameplay.Get_Score(1,state_prime)
            agent.memorize(state,action,reward,state_prime,status)


    if turn % 2 ==1:
        action2=bot.Rando_bot(state)
        #action2=agent2.make_move(state)
        #action2=bot.MiniMaxAlphaBeta_bot(state,2)
        gameplay.Add_Piece(2,action2,gameplay.BOARD) #do action for other player
        state_prime = gameplay.BOARD.copy() #get state prime 
        if action not in gameplay.Get_Legal_Moves(state): #punish agent for trying to make an illegal move
            reward=-5
            state_prime = state.copy()
        else:
            reward = gameplay.Get_Score(1,state_prime) #get reward for bot
        status=gameplay.Check_Goal(gameplay.BOARD) #get status
        
        agent.memorize(state,action,reward,state_prime,status) #memorize state/reward/stateprime/status


    #check if it is the end of the episode, then reset (either status change or threshold moves played)
    if status !='Keep Playing!' or turn>threshold:
        if status == 'Player 1 wins!':
            p1wins=p1wins+1
        if status == 'Player 2 wins!':
            p2wins=p2wins+1
        
        #reset game
        turn=0
        gameplay.reset()
        episode=episode+1
        
        loss=agent.replay(agent.batch_size) # do the thing
        agent2.epsilon=agent.epsilon #update agent2
        agent.save('weights.h5')
        agent2.load('weights.h5') # load weights of the updated agent for player 2


    turn = turn + 1 #next turn
    
    #the prints and stuff
    if episode % 1000==0 and status !='Keep Playing!':
        print('--------------------------------------------------------')
        print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
        print('EPISODE: ',episode)
        print('Avg loss last 1000:',round(sum(agent.losses)/(len(agent.losses)+.001),2),'epsilon:',round(agent.epsilon,3))
        print('Avg reward:',round(sum(agent.avgrewards)/(len(agent.avgrewards)+.001),2))
        
        agent.losses=[]
        agent.avgrewards=[]
        winrate=round((p1wins-p1prev)/(p1wins+p2wins-p1prev-p2prev+.001),2)
        print('player 1 wins:',p1wins,'player 2 wins:',p2wins, 'winrate:',winrate)
        p1prev=p1wins
        p2prev=p2wins

    if episode==100 and status !='Keep Playing!':
        print('EPISODE:',episode,'epsilon:',round(agent.epsilon,3))
        print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
        keras.models.save_model(agent.model,'mymodel_'+str(episode)+'.h5')
        
    if episode==10000 and status !='Keep Playing!':
        print('EPISODE:',episode,'epsilon:',round(agent.epsilon,3))
        print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
        keras.models.save_model(agent.model,'mymodel_'+str(episode)+'.h5')
    if episode==20000 and status !='Keep Playing!':
        print('EPISODE:',episode,'epsilon:',round(agent.epsilon,3))
        print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
        keras.models.save_model(agent.model,'mymodel_'+str(episode)+'.h5')
    if episode==50000 and status !='Keep Playing!':
        print('EPISODE:',episode,'epsilon:',round(agent.epsilon,3))
        print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
        keras.models.save_model(agent.model,'mymodel_'+str(episode)+'.h5')
        
    if episode % 100000==0 and status !='Keep Playing!':
        print('EPISODE:',episode,'epsilon:',round(agent.epsilon,3))
        print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
        keras.models.save_model(agent.model,'mymodel_'+str(episode)+'.h5')


In [76]:
for mem in agent.memory:
    print('MEMORY:')
    print(mem[0])
    print('ACTION:',mem[1],'REWARD:',mem[2])
    print('Qs:')
    print(agent.model_predict(mem[0]))
    print('pred:',np.argmax(agent.model_predict(mem[0])))

MEMORY:
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  0. -1. -1.  0.  0.]]
ACTION: 5 REWARD: 0.25
Qs:
[[-2.4810693 -2.5444367 -1.474736  -2.645671  -2.377594  -2.0255926
  -2.9881983]]
pred: 2
MEMORY:
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -1.  0.]
 [ 0.  1.  0. -1.  0. -1.  0.]
 [ 1.  1. -1.  1. -1. -1.  0.]
 [ 1.  1.  1. -1. -1.  1.  0.]]
ACTION: 3 REWARD: -0.75
Qs:
[[-2.8857534 -4.0283165 -1.1920003 -4.230981  -4.8621635 -1.8137944
  -3.4866014]]
pred: 2
MEMORY:
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  1.  0.  0.  0.]]
ACTION: 4 REWARD: 1.25
Qs:
[[-0.78141797 -1.5069183  -0.63121045 -0.40267828  0.0372297  -1.2855872
  -1.17481   ]]
pred: 4
MEMORY:
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  

C:\Users\wamsa\Anaconda3\lib\site-packages\ipykernel_launcher.py:114: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.


pred: 4
MEMORY:
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0. -1. -1. -1.  0.]
 [ 0.  0.  0. -1.  1. -1.  0.]
 [ 1.  0.  0. -1.  1.  1.  0.]
 [ 1.  0.  0.  1.  1. -1. -1.]]
ACTION: 3 REWARD: -0.75
Qs:
[[-2.7082512 -4.6455913 -1.8734235 -5.106502  -3.8728323 -3.8680334
  -6.9673934]]
pred: 2
MEMORY:
[[ 0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  1.  0. -1.  0.]
 [ 0.  0.  0. -1. -1. -1.  0.]
 [ 1.  0.  0. -1.  1. -1.  0.]
 [ 1.  0.  0. -1.  1.  1.  0.]
 [ 1. -1.  0.  1.  1. -1. -1.]]
ACTION: 4 REWARD: -1.5
Qs:
[[-1.9321607  -5.9095364  -0.84055203 -3.8652787  -4.592725   -3.5506787
  -5.284964  ]]
pred: 2
MEMORY:
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.]
 [-1.  1. -1. -1.  1.  1. -1.]]
ACTION: 6 REWARD: 1.75
Qs:
[[-1.339597  -1.9502689 -1.446936  -1.0955474 -0.3590261 -1.9070122
  -2.503065 ]]
pred: 4
MEMORY:
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  

[[-4.4632773 -4.237311  -2.2465336 -3.165017  -4.198529  -4.3684144
  -5.4427633]]
pred: 2
MEMORY:
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  1. -1.  0. -1. -1.]]
ACTION: 3 REWARD: -50
Qs:
[[-1.7934859  -1.498861   -0.6918976  -2.1563487  -0.93463844 -0.7760475
  -2.229115  ]]
pred: 2
MEMORY:
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  1. -1.  0. -1. -1.]]
ACTION: 3 REWARD: -1.75
Qs:
[[-1.7934859  -1.498861   -0.6918976  -2.1563487  -0.93463844 -0.7760475
  -2.229115  ]]
pred: 2
MEMORY:
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. -1.  0.  0.]
 [ 0.  0.  0.  1. -1.  0.  0.]
 [ 1. -1.  0. -1. -1.  1.  1.]]
ACTION: 3 REWARD: -1.25
Qs:
[[-4.670171  -3.8005915 -2.2717404 -3.5159738 -3

pred: 2
MEMORY:
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. -1.  0.  0.]
 [ 0.  0.  0.  0. -1.  0.  0.]
 [ 1.  0.  1. -1. -1.  1.  0.]]
ACTION: 4 REWARD: -1.5
Qs:
[[-5.1819987 -4.780803  -2.6994402 -4.3242097 -4.9166026 -3.4228284
  -5.2011886]]
pred: 2
MEMORY:
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0. -1.  0.  0.]
 [ 1.  0. -1.  0. -1.  0.  1.]
 [ 1. -1.  1. -1. -1.  1. -1.]]
ACTION: 3 REWARD: -1.0
Qs:
[[-3.5667827 -4.0896134 -1.3369988 -5.4424424 -5.6728435 -2.2975676
  -5.570394 ]]
pred: 2
MEMORY:
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. -1.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0. -1.  0. -1.]
 [ 1.  0. -1.  1. -1.  1.  1.]
 [ 1. -1.  1. -1. -1.  1. -1.]]
ACTION: 1 REWARD: -1.25
Qs:
[[-3.799624  -4.828444  -1.5129625 -6.1880965 -6.0716586 -2.3719423
  -5.8900046]]
pred: 2
MEMORY:
[[ 0.  0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.

...

ACTION: 1 REWARD: -0.75
Qs:
[[-5.0624533 -4.5858355 -3.287505  -1.1149545 -0.8040785 -5.0788236
  -3.216156 ]]
pred: 4
MEMORY:
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 2. 2. 0. 0. 2.]]
ACTION: 5 REWARD: -1.5
Qs:
[[-4.923205   -4.295577   -2.3083956  -0.9332511   0.49395823 -4.833579
  -2.3576922 ]]
pred: 4
MEMORY:
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 2. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 2. 2. 0. 1. 2.]]
ACTION: 0 REWARD: -2.0
Qs:
[[-2.7695558 -2.4213998 -1.5474012 -0.5151534  0.6963486 -3.6631722
  -1.3242248]]
pred: 4
MEMORY:
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 2. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 2.]
 [1. 1. 2. 2. 0. 1. 2.]]
ACTION: 5 REWARD: -1.75
Qs:
[[-3.5462024 -3.230819  -1.2043194 -1.3969815 -1.6083474 -4.369277
  -2.7332234]]
pred: 2
MEMORY:
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0

pred: 4
MEMORY:
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 2. 0. 0. 0. 0. 0.]
 [0. 1. 2. 0. 2. 0. 1.]
 [0. 1. 1. 0. 2. 1. 2.]
 [0. 1. 1. 0. 2. 2. 2.]
 [1. 2. 2. 0. 1. 1. 2.]]
ACTION: 3 REWARD: -50
Qs:
[[-5.382389  -2.7630134 -3.0335045 -2.808989  -1.1533469 -5.1223803
  -4.805076 ]]
pred: 4
MEMORY:
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 2. 0. 0. 0. 0. 0.]
 [0. 1. 2. 0. 2. 0. 1.]
 [0. 1. 1. 0. 2. 1. 2.]
 [0. 1. 1. 0. 2. 2. 2.]
 [1. 2. 2. 0. 1. 1. 2.]]
ACTION: 3 REWARD: -0.75
Qs:
[[-5.382389  -2.7630134 -3.0335045 -2.808989  -1.1533469 -5.1223803
  -4.805076 ]]
pred: 4
MEMORY:
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.]]
ACTION: 6 REWARD: -1.0
Qs:
[[-0.5866834  -0.63385135  0.22351299 -0.18692547  0.53011113 -0.656272
  -1.0228806 ]]
pred: 4
MEMORY:
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0.

[[-5.2917557 -6.1673865 -3.871925  -0.0214913  1.3338301 -8.077615
  -3.9906535]]
pred: 4
MEMORY:
[[0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 2. 0. 0.]
 [0. 0. 0. 0. 2. 0. 0.]
 [2. 0. 0. 0. 1. 2. 0.]
 [1. 1. 2. 0. 2. 1. 0.]
 [1. 1. 2. 0. 2. 2. 1.]]
ACTION: 2 REWARD: -2.0
Qs:
[[-4.0709634  -6.2788634  -2.785205   -0.22459574 -0.31133857 -7.596698
  -3.63771   ]]
pred: 3
MEMORY:
[[0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 2. 0. 0.]
 [0. 0. 0. 0. 2. 2. 0.]
 [2. 0. 1. 0. 1. 2. 0.]
 [1. 1. 2. 0. 2. 1. 0.]
 [1. 1. 2. 0. 2. 2. 1.]]
ACTION: 3 REWARD: -1.0
Qs:
[[-3.0611165  -6.0315366  -0.52749926 -0.32244834 -1.0428774  -7.2971625
  -3.7390332 ]]
pred: 3
MEMORY:
[[0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 2. 0. 0.]
 [0. 0. 0. 0. 2. 2. 0.]
 [2. 0. 1. 0. 1. 2. 0.]
 [1. 1. 2. 2. 2. 1. 0.]
 [1. 1. 2. 1. 2. 2. 1.]]
ACTION: 2 REWARD: 0.5
Qs:
[[-2.7621117  -4.9854803  -0.7804378  -0.10281233 -0.3818409  -5.8395243
  -2.7948217 ]]
pred: 3
MEMORY:
[[ 0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0. -1. -1.  0.]
 [ 0.  0.  

 [ 0.  0. -1.  0.  0. -1.  0.]]
ACTION: 0 REWARD: -0.75
Qs:
[[-1.9101537 -2.6519136 -1.5447562 -1.533393  -1.7229924 -2.393
  -3.017819 ]]
pred: 3
MEMORY:
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 2. 0.]
 [2. 0. 1. 0. 0. 1. 0.]
 [1. 0. 2. 0. 0. 2. 0.]]
ACTION: 2 REWARD: 0.75
Qs:
[[-5.441505  -6.0646653 -4.85792   -4.5217266 -4.107489  -4.8781967
  -5.955554 ]]
pred: 4
MEMORY:
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [2. 0. 1. 0. 0. 2. 0.]
 [2. 0. 1. 0. 0. 1. 0.]
 [1. 0. 2. 0. 0. 2. 0.]]
ACTION: 1 REWARD: 0.0
Qs:
[[-3.3108156 -5.650074  -2.707076  -2.9142413 -3.6791952 -4.8490767
  -4.367883 ]]
pred: 2
MEMORY:
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 2. 0.]
 [2. 0. 1. 0. 0. 2. 0.]
 [2. 0. 1. 0. 0. 1. 0.]
 [1. 1. 2. 0. 0. 2. 0.]]
ACTION: 3 REWARD: -1.0
Qs:
[[-3.6342535 -6.320126  -2.6423712 -2.8131416 -4.4208474 -4.861465
  -4.470011 ]]
pred: 2
MEMORY:
[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 

[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 2. 0. 0. 0. 0.]
 [2. 0. 2. 0. 0. 0. 2.]
 [2. 1. 1. 2. 1. 1. 1.]]
ACTION: 6 REWARD: -5.75
Qs:
[[-7.0546947 -6.175652  -1.6872554 -0.0410264 -2.439316  -8.412851
  -6.259477 ]]
pred: 3
MEMORY:
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 2. 0. 0. 0. 1.]
 [2. 2. 2. 0. 0. 0. 2.]
 [2. 1. 1. 2. 1. 1. 1.]]
ACTION: 4 REWARD: -5.25
Qs:
[[-6.761906   -6.8419137  -0.84371847 -0.1804276  -2.9952652  -7.4044447
  -5.827518  ]]
pred: 3
MEMORY:
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 2. 2. 0. 0. 0. 1.]
 [2. 2. 2. 0. 1. 0. 2.]
 [2. 1. 1. 2. 1. 1. 1.]]
ACTION: 3 REWARD: -3.75
Qs:
[[-3.9699078 -4.189881   1.0605245 -1.1084841 -3.2901201 -5.3117623
  -4.85499  ]]
pred: 2
MEMORY:
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 2.]
 [0. 2. 2. 0. 0. 0. 1.]
 [2. 2. 2. 1. 1. 0. 2.]
 [2. 1. 1. 2. 1. 1. 1.]]
ACTION: 2 REWARD: -1.25
Qs:
[[-2.7

In [70]:
agent.memory

[(array([[1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.]]),
  1,
  1.0,
  array([[0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0.]]),
  'Keep Playing!'),
 (array([[ 1.,  1.,  1.,  1.,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.,  1.,  1.,  1.],
         [ 1., -1.,  1.,  1.,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  1.,  1.,  1.,  1.]]),
  0,
  0.25,
  array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0., -1.,  

In [ ]:
agent.losses

In [ ]:
        for el in batch:
            if self.player==1: #replace 2's with -1s
                el[0][el[0]>1]=-1
                el[3][el[3]>1]=-1
            if self.player==2: #replace 1s with -1s. replace 2's with 1s
                el[0][(el[0] < 2) & (el[0] > 0)] = -1
                el[0][el[0]>1]=1                
                el[3][(el[3] < 2) & (el[3] > 0)] = -1
                el[3][el[3]>1]=1                
        
        states = []
        targets_f = []

In [ ]:
states = []
targets_f = []
batch = random.sample(agent.memory,agent.batch_size)
for state,action, reward, state_prime,status in batch:
    if status != 'Keep Playing!':
        target = reward
    elif status == 'Keep Playing!':
        if action not in gameplay.Get_Legal_Moves(state): #if not a legal move make target the negative reward
            target = reward
        else:
             target = (reward + agent.gamma * np.argmax(agent.model_predict(state_prime))) #idk why they have first element.

    target_f = agent.model_predict(state_prime)
    print(target_f[0][action])
    targets_f.append(target_f[0][action])
    states.append(state)
    #print('STATE:',state)
    #print('ACTION:',action)
    #print('target:',target)
    #print('reward:',reward)

            
states=np.expand_dims(np.array(states), axis=3) #reshaping to train model
targets_f=np.array(targets_f)


In [ ]:
keras.models.save_model(agent.model,'mymodel_'+str(episode)+'.h5')

In [ ]:
agent.memory

In [ ]:
from tensorflow.keras.models import Sequential, save_model, load_model
keras.models.save_model(agent.model,'mymodel.h5')

In [ ]:
agent.memory[-1][2]

In [ ]:
agent.memory[1][0]

In [ ]:
gameplay.BOARD

In [ ]:
if status != 'Keep Playing!': #if we are in a terminal state restart game
    self.gameplay.reset()
    turn = 0

state = self.gameplay.BOARD.copy()
actions = self.gameplay.Get_Legal_Moves(self.gameplay.BOARD)
action = np.random.choice(actions,1)[0]
#take random actions. record their states,actions, rewards, next states, and status

    #player1
if turn % 2 ==0:
    self.gameplay.Add_Piece(1,action,self.gameplay.BOARD)
        #turn +=1

    #player2
elif turn % 2 ==1:
    self.gameplay.Add_Piece(2,action,self.gameplay.BOARD)
        #turn +=1


status = self.gameplay.Check_Goal(self.gameplay.BOARD)
if status !='Keep Playing!':
    state_prime = np.zeros((self.gameplay.ROWS,self.gameplay.COLUMNS))
    state_prime[state_prime < 1] = 1
else:
    state_prime = self.gameplay.BOARD.copy() #gamestate
reward = self.gameplay.get_reward(self.player,state)

if turn % 2 ==0 and self.player==1:
    memory.append((state,action,reward,state_prime,status)) #memorize this
if turn % 2 ==1 and self.player==2:
    memory.append((state,action,reward,state_prime,status)) #memorize this
turn +=1


In [ ]:

keras.models.save_model(
agent1.model,'mymodel.h5')

In [ ]:
#saving model and using it for predictions
from tensorflow.keras.models import Sequential, save_model, load_model

keras.models.save_model(
agent.model,'mymodel.h5')

loaded_model = load_model('mymodel.h5')
state=np.expand_dims(np.expand_dims(gameplay.BOARD, axis=3),axis=0)
loaded_model.predict(state)

In [ ]:
state=np.expand_dims(np.expand_dims(gameplay.BOARD, axis=3),axis=0)
loaded_model.predict(state)

In [ ]:
import keras
from keras import layers
from keras import models

model=models.Sequential()
model.add(layers.Conv2D(16, (3,3), activation='relu', input_shape=INPUT_SHAPE))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Flatten())
model.add(layers.Dense(32,activation='relu'))
model.add(layers.Dense(action_size,activation='softmax'))

model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])
model.summary()